### Packages

In [ ]:
###Packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.metrics import make_scorer, mean_absolute_error
# to install below skopt package, write in terminal:
# pip install scikit-optimize
from skopt import BayesSearchCV
from skopt.space import Real, Categorical, Integer
import pickle

## Data Import

In [ ]:
df = pd.read_csv("DataTrain-2.csv")

#### Check for null values and infinite values

In [ ]:
# Checking which columns have any null values
columns_with_nulls = df.isnull().any()

# Filtering to show only columns with null values
columns_with_nulls_true = columns_with_nulls[columns_with_nulls].index.tolist()

print("Columns with null values:", columns_with_nulls_true)

# Check for columns with infinite values (positive or negative)
infinite_columns = df.isin([np.inf, -np.inf]).any()

# Filter and print column names that have infinite values
columns_with_infinity = infinite_columns[infinite_columns].index.tolist()
print("Columns with infinite values:", columns_with_infinity)

#### Create a new column RUL 

In [ ]:
# Creating a new column with the max 'cycle' per 'engine_id'
df['max_cycle_per_engine'] = df.groupby('engine_id')['cycle'].transform('max')
# Creating the 'cycles_left' column
df['RUL'] = df['max_cycle_per_engine'] - df['cycle']
df.drop(['max_cycle_per_engine'], axis=1, inplace=True)

### Remove features with zero variance

In [ ]:
# Calculating variance for each column
variances = df.var()

# Identifying columns with zero variance
zero_variance_columns = variances[variances == 0].index.tolist()

print("Columns with zero variance:", zero_variance_columns)

# Filtering out columns with zero variance
df = df.loc[:, variances > 0]

#### Remove the columns with one unique value

In [ ]:
# Checking the number of unique values per column
unique_counts = df.nunique()

# Identifying columns with only one unique value
columns_to_remove = unique_counts[unique_counts == 1].index.tolist()

# Removing these columns from the DataFrame
df.drop(columns=columns_to_remove, inplace=True)

# Printing the names of the columns that were removed
print("Columns removed:", columns_to_remove)

## Correlation matrix

In [ ]:
# Calculating the correlation matrix
correlation_matrix = df.corr()

# Display the correlation matrix
print(correlation_matrix)

# Visualizing the correlation matrix using seaborn
plt.figure(figsize=(8, 6))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f", linewidths=.5)
plt.show()

In [ ]:
rul_correlations = correlation_matrix['RUL']

# Display the correlations of 'RUL' with other features
print(rul_correlations)

In [ ]:
df.columns

## Add lag features

In [ ]:
# Calculate mean values across all engines for each cycle
mean_df = df.groupby('cycle').mean().reset_index()
import matplotlib.pyplot as plt
import seaborn as sns

def plot_mean_sensors(df, mean_df):
    sns.set(style="whitegrid")  # Set the aesthetic style of the plots

    # Define columns that are not to be included in plotting (non-sensor columns)
    exclude_columns = ['engine_id', 'cycle']
    sensor_columns = [col for col in df.columns if col not in exclude_columns]

    for sensor in sensor_columns:
        plt.figure(figsize=(12, 6))  # Adjust the size of the figure for better visibility

        # Plotting each engine's sensor data (you can remove this part if you want only the mean trendline)
        sns.lineplot(data=df, x='cycle', y=sensor, hue='engine_id', palette='tab10', legend=None)
        
        # Plotting the mean trendline
        sns.lineplot(data=mean_df, x='cycle', y=sensor, color='red', linewidth=2.5, label='Mean Trend')

        plt.title(f'Sensor Readings Over Cycles for {sensor}')
        plt.xlabel('Cycle')
        plt.ylabel(sensor)
        # plt.legend(title='Engine ID', bbox_to_anchor=(1.05, 1), loc='upper left')  # Removed as per requirement
        plt.show()

# Assuming your DataFrame is ready and named 'df'
plot_mean_sensors(df, mean_df)

In [ ]:
# define a function to create difference features that capture the rate of change in sensor readings compared to the first cycle

def create_difference_features(df, sensor_columns):
    # Create a copy of the DataFrame to avoid modifying the original data
    df_diff = df.copy()

    # Create difference features for each sensor column
    for sensor in sensor_columns:
        # Create a new column name for the difference feature
        diff_col = f'{sensor}_diff'

        # Calculate the difference values
        df_diff[diff_col] = df_diff.groupby('engine_id')[sensor].transform(lambda group: group - group.iloc[0])

    return df_diff

# Define the sensor columns to create difference features for
sensor_columns = ['sensor_val2', 'sensor_val4', 'sensor_val5', 'sensor_val6',
                  'sensor_val9', 'sensor_val10', 'sensor_val12', 'sensor_val13', 'sensor_val15', 'sensor_val17', 'sensor_val19', 'sensor_val21']

# Create difference features for the selected sensor columns
df_diff = create_difference_features(df, sensor_columns)

# Only keep difference features and the 'engine_id' and 'cycle' columns
df_diff_clean = df_diff[['engine_id', 'cycle'] + [col for col in df_diff.columns if 'diff' in col]]

# Left join the difference features with the original DataFrame on 'engine_id' and 'cycle'
df = df.merge(df_diff_clean, on=['engine_id', 'cycle'], how='left')

# Display the resulting DataFrame
print(df.head())

In [ ]:
df.head(100)

## Modeling linear regression

In [ ]:
# Defining the dependent variable and independent variables
X = df[['cycle', 'set2', 'set3', 'sensor_val1', 'sensor_val2',
       'sensor_val3', 'sensor_val4', 'sensor_val5', 'sensor_val6',
       'sensor_val9', 'sensor_val10', 'sensor_val12', 'sensor_val13',
       'sensor_val15', 'sensor_val17', 'sensor_val18', 'sensor_val19',
       'sensor_val21', 'sensor_val2_diff', 'sensor_val4_diff', 'sensor_val5_diff', 'sensor_val6_diff', 'sensor_val9_diff', 'sensor_val10_diff', 'sensor_val12_diff', 'sensor_val13_diff', 'sensor_val15_diff', 'sensor_val17_diff', 'sensor_val19_diff', 'sensor_val21_diff']]    # possible to add lag RUL
y = df['RUL']

# Adding a constant to the model (intercept)
X = sm.add_constant(X)

# Setting up the OLS model
model = sm.OLS(y, X)

# Fitting the model
results = model.fit()

# Printing the summary of the model
print(results.summary())

eventhough set 2 had a higher correlation with RUL than set 3, controlling for other variables revealed that set 2 is not significant

remove set 2 as a feature, and drop other non-significant features

In [ ]:
# Defining the dependent variable and independent variables
X = df[['cycle', 'set3', 'sensor_val1', 'sensor_val2', 'sensor_val4', 'sensor_val5', 'sensor_val6', 'sensor_val10', 'sensor_val12', 'sensor_val13',
       'sensor_val15', 'sensor_val17', 'sensor_val1',
       'sensor_val21', 'sensor_val2_diff', 'sensor_val5_diff', 'sensor_val6_diff', 'sensor_val9_diff', 'sensor_val13_diff', 'sensor_val15_diff', 'sensor_val17_diff', 'sensor_val19_diff', 'sensor_val21_diff']] 

In [ ]:
# Define the dependent variable and independent variables
y = df['RUL']

# Splitting the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

# Creating the Random Forest model
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)

# Fitting the model
rf_model.fit(X_train, y_train)

# Applying 5-fold cross-validation
cv_scores = cross_val_score(rf_model, X_train, y_train, cv=5, scoring=make_scorer(mean_absolute_error))

# Output the results of cross-validation
print("CV Scores:", cv_scores)
print("CV Average Score:", np.mean(cv_scores))

In [ ]:
# Random Forest model setup
rf = RandomForestRegressor(random_state=42)

# Define parameter space for Bayesian Optimization
search_spaces = {
    'n_estimators': Integer(50, 200),
    'max_depth': Integer(10, 30),
    'min_samples_split': Integer(2, 10),
    'min_samples_leaf': Integer(1, 4)
}

# Setting up Bayesian Search
opt = BayesSearchCV(estimator=rf, search_spaces=search_spaces, n_iter=32, scoring='neg_mean_absolute_error', n_jobs=-1, cv=5, verbose=2, random_state=42)

# Fitting Bayesian Search
opt.fit(X_train, y_train)

# Best model evaluation
print("Best parameters:", opt.best_params_)
print("Best score (neg_mean_absolute_error):", opt.best_score_)

# Optionally, check the performance on the test set
best_rf = opt.best_estimator_
best_rf.fit(X_train, y_train)
test_score = best_rf.score(X_test, y_test)
print("Test set score:", test_score)

In [ ]:
# Specify the file path to save the model
model_file_path = 'best_random_forest_model.pkl'

# Save the model to disk
with open(model_file_path, 'wb') as file:
    pickle.dump(best_rf, file)

print(f"Model saved to {model_file_path}")

In [ ]:
df = pd.read_csv("DataSchedule-2.csv")

# Create difference features for the selected sensor columns
df_diff = create_difference_features(df, sensor_columns)

# Only keep difference features and the 'engine_id' and 'cycle' columns
df_diff_clean = df_diff[['engine_id', 'cycle'] + [col for col in df_diff.columns if 'diff' in col]]

# Left join the difference features with the original DataFrame on 'engine_id' and 'cycle'
df = df.merge(df_diff_clean, on=['engine_id', 'cycle'], how='left')

# Specify the path to the pickled model file
model_file_path = 'best_random_forest_model.pkl'
# Load the model from disk
with open(model_file_path, 'rb') as file:
    model = pickle.load(file)
df_features = df[['cycle', 'set3', 'sensor_val1', 'sensor_val2', 'sensor_val4', 'sensor_val5', 'sensor_val6', 'sensor_val10', 'sensor_val12', 'sensor_val13',
       'sensor_val15', 'sensor_val17', 'sensor_val1',
       'sensor_val21', 'sensor_val2_diff', 'sensor_val5_diff', 'sensor_val6_diff', 'sensor_val9_diff', 'sensor_val13_diff', 'sensor_val15_diff', 'sensor_val17_diff', 'sensor_val19_diff', 'sensor_val21_diff']] 

In [ ]:
#Making predictions and round to the nearest integer
predictions = np.round(model.predict(df_features))

#Converting to integer type
predictions = predictions.astype(int)
predictions

## add the predictions to df
df['RUL'] = predictions
df
## Take the final row of each engine id
# Group by 'engine_id' and get the last 'RUL' value for each engine
last_rul_per_engine = df.groupby('engine_id')['RUL'].last()
last_rul_per_engine_df = last_rul_per_engine.reset_index()
# Combine 'RUL' and 'engine_id' into a single column formatted as "RUL;id"
last_rul_per_engine_df['RUL;id'] = last_rul_per_engine_df['RUL'].astype(str) + ';' + last_rul_per_engine_df[
    'engine_id'].astype(str)

In [ ]:
# Create a new DataFrame with only the combined column
result_df = last_rul_per_engine_df[['RUL;id']]

# Save the DataFrame to a CSV file
result_df.to_csv('rul_id_pairs.csv', index=False, header=True)

In [ ]:
# Compare to the consultancy predictions
df_our_predictions = pd.read_csv('rul_id_pairs.csv')
df_our_predictions["RULours;engine_id"] = df_our_predictions["RUL;id"]
df_our_predictions.drop(['RUL;id'], axis=1, inplace=True)
df_consultancy = pd.read_csv("RUL_consultancy_predictions_A3-2.csv")
df_consultancy["RULconsul;engine_id"] = df_consultancy["RUL;id"]
df_consultancy.drop(['RUL;id'], axis=1, inplace=True)
df_combined = pd.concat([df_our_predictions, df_consultancy], axis=1)
df_combined
# Split the 'RUL;id' column into two new columns
df_combined[['RULours', 'engine_id']] = df_combined['RULours;engine_id'].str.split(';', expand=True)
df_combined[['RULconsul', 'old_id']] = df_combined['RULconsul;engine_id'].str.split(';', expand=True)

In [ ]:
# Drop the original columns
df_combined.drop(['RULours;engine_id', 'RULconsul;engine_id'], axis=1, inplace=True)

# drop the old_id
df_combined.drop(['old_id'], axis=1, inplace=True)

#Convert these columns to an appropriate type (e.g., int)
df_combined['RULours'] = df_combined['RULours'].astype(float)
df_combined['RULconsul'] = df_combined['RULconsul'].astype(float)
df_combined['engine_id'] = df_combined['engine_id'].astype(int)
df_combined
# Plotting histograms
plt.figure(figsize=(10, 6))  # Set the figure size for better readability

# Histogram for RULours
plt.hist(df_combined['RULours'], bins=10, alpha=0.5, label='RULours', color='blue')

# Histogram for RULconsul
plt.hist(df_combined['RULconsul'], bins=10, alpha=0.5, label='RULconsul', color='red')

# Adding labels and title
plt.xlabel('Remaining Useful Life')
plt.ylabel('Frequency')
plt.title('Histogram of RULours and RULconsul')
plt.legend(loc='upper right')

# Show plot
plt.show()
# Calculate the difference between RULours and RULconsul
df_combined['Difference'] = df_combined['RULours'] - df_combined['RULconsul']

# Plotting the histogram of the differences
plt.figure(figsize=(8, 6))  # Set the figure size for better readability
plt.hist(df_combined['Difference'], bins=10, color='green', alpha=0.7)
plt.xlabel('Difference in RUL (RULours - RULconsul)')
plt.ylabel('Frequency')
plt.title('Histogram of Difference between RULours and RULconsul')
plt.grid(True)  # Optional: adds a grid for easier readability
plt.show()

In [ ]:
def bootstrap_diff(data, n_bootstrap=1000):
    bootstrap_means = []
    for _ in range(n_bootstrap):
        # Sample with replacement from the data
        sample = data.sample(n=len(data), replace=True)
        # Calculate the mean difference
        mean_diff = sample['RULours'].mean() - sample['RULconsul'].mean()
        bootstrap_means.append(mean_diff)
    return bootstrap_means


# Performing bootstrap sampling
diffs = bootstrap_diff(df_combined)

# Plotting the distribution of bootstrap differences
plt.hist(diffs, bins=30, color='blue', alpha=0.7)
plt.xlabel('Mean Difference (RULours - RULconsul)')
plt.ylabel('Frequency')
plt.title('Bootstrap Distribution of Mean Differences')
plt.axvline(x=0, color='red', linestyle='--')
plt.show()
# Calculating the 95% confidence interval
conf_interval = np.percentile(diffs, [2.5, 97.5])
print("95% Confidence Interval for the Mean Difference:", conf_interval)